In [1]:
import os
import SimpleITK as sitk
from lxml import etree
import json
import shutil

In [2]:
input_directory = r'C:\Users\lydia\OneDrive\Work\DigitalAssets\reference_models'
output_directory = r'C:\Users\lydia\OneDrive\Work\DigitalAssets\atlas_models'

In [3]:
tlist = ['E11.5','E13.5','E15.5','E16.5','E18.5','P4','P14','P28','P56']
plist = ['sagittal','coronal']

atlas_metadata = {}

In [4]:
for t in tlist:
    
    atlas_metadata[t] = {}
    
    for p in plist:
        
        print((t,p))
        
        input_sub_directory = os.path.join( input_directory, t, p )
        output_sub_directory = os.path.join( output_directory, t, p )
        os.makedirs( output_sub_directory, exist_ok = True )
        
        atlas_metadata[t][p] = {}
        
        atlas_metadata[t][p]['standard_size'] = {}
        
        xml_file = os.path.join( input_sub_directory, 'model.xml')
        xt = etree.parse(xml_file)
        root = xt.getroot()
        
        el = root.find("output-width")
        atlas_metadata[t][p]['standard_size']['width'] = float(el.text)
        
        el = root.find("output-height")
        atlas_metadata[t][p]['standard_size']['height'] = float(el.text)
        
        input_file = os.path.join( input_sub_directory, 'atlasVolume.mhd')
        atlas = sitk.ReadImage( input_file )
        
        output_file = os.path.join( output_sub_directory, 'atlasVolume.nii.gz')
        sitk.WriteImage( atlas, output_file, True )
        
        atlas_metadata[t][p]['atlas'] = {}
        atlas_metadata[t][p]['atlas']['path'] = '%s/%s/%s' % (t,p,'atlasVolume.nii.gz')
        atlas_metadata[t][p]['atlas']['size'] = atlas.GetSize()
        atlas_metadata[t][p]['atlas']['spacing'] = atlas.GetSpacing()
        atlas_metadata[t][p]['atlas']['origin'] = atlas.GetOrigin()
        
        input_file = os.path.join( input_sub_directory, 'atlasVolumeToGrid.meta' )
        output_file = os.path.join( output_sub_directory, 'CanonicalToAtlasVolume.tfm')
        shutil.copy( input_file, output_file )
        
        xform = sitk.ReadTransform( output_file )
        
        atlas_metadata[t][p]['canonical_to_atlas_transform'] = {}
        atlas_metadata[t][p]['canonical_to_atlas_transform']['transform_type']= 'CenteredAffineTransform'
        atlas_metadata[t][p]['canonical_to_atlas_transform']['parameters'] = xform.GetParameters()
        atlas_metadata[t][p]['canonical_to_atlas_transform']['fixed_parameters'] = xform.GetFixedParameters()
        

('E11.5', 'sagittal')
('E11.5', 'coronal')
('E13.5', 'sagittal')
('E13.5', 'coronal')
('E15.5', 'sagittal')
('E15.5', 'coronal')
('E16.5', 'sagittal')
('E16.5', 'coronal')
('E18.5', 'sagittal')
('E18.5', 'coronal')
('P4', 'sagittal')
('P4', 'coronal')
('P14', 'sagittal')
('P14', 'coronal')
('P28', 'sagittal')
('P28', 'coronal')
('P56', 'sagittal')
('P56', 'coronal')


In [5]:
atlas_metadata

{'E11.5': {'sagittal': {'standard_size': {'width': 8832.0, 'height': 9497.6},
   'atlas': {'path': 'E11.5/sagittal/atlasVolume.nii.gz',
    'size': (345, 371, 158),
    'spacing': (16.0, 16.0, 20.0),
    'origin': (0.0, 0.0, -160.0)},
   'canonical_to_atlas_transform': {'transform_type': 'CenteredAffineTransform',
    'parameters': (1.0,
     0.0,
     0.0,
     0.0,
     1.0,
     0.0,
     0.0,
     0.0,
     1.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0),
    'fixed_parameters': (0.0, 0.0, 0.0)}},
  'coronal': {'standard_size': {'width': 9600.0, 'height': 9600.0},
   'atlas': {'path': 'E11.5/coronal/atlasVolume.nii.gz',
    'size': (600, 600, 309),
    'spacing': (16.0, 16.0, 20.0),
    'origin': (0.0, 0.0, 20.0)},
   'canonical_to_atlas_transform': {'transform_type': 'CenteredAffineTransform',
    'parameters': (-0.159816,
     0.0151561,
     -1.00399,
     0.437843,
     1.07334,
     -0.0961956,
     -1.06962,
     0.408105,
     0.315819,
     2752.0,
     296

In [6]:
output_file = os.path.join( output_directory, 'atlas_metadata.json')
with open( output_file, 'w') as mfile :
        json.dump( atlas_metadata, mfile, indent=4 )